In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import pandas as pd
import numpy as np
from pydantic import BaseModel
import sys
import os
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from load_dotenv import load_dotenv
import time
import json
import torch
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow

sys.path.insert(0, "..")

from src.utils.embedding_id_mapper import IDMapper
from src.algo.gSASRec.model import SASRec
from src.algo.gSASRec.dataset import SASRecDataset
from src.algo.gSASRec.trainer import SASRecLitModule
from src.eval.utils import create_rec_df, create_label_df, merge_recs_with_target
from src.eval.log_metrics import log_ranking_metrics, log_classification_metrics

In [3]:
load_dotenv(override = True)

False

In [4]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "first-attempt"
    run_name: str = f"018-sasrec"
    notebook_persit_dp: str = None
    
    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    group_name: str = "seq-modelling"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 16
    lr: float = 0.0001
    l2_emb: float = 0.0001
    early_stopping_patience: int = 10
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    num_epochs: int = 100

    # SASrec specific
    max_len: int = 10
    dropout: float = 0.5
    hidden_units: int = 128
    num_blocks: int = 1
    num_heads: int = 2
    num_workers: int = 4
    gbce_t: int = 0.0
    pad_token: int = 0
    # seq_length: int = 10
    
    train_data_fp: str = os.path.abspath("../data_for_ai/interim/train_sample_interactions_16407u_neg_seq.parquet")
    val_data_fp: str = os.path.abspath("../data_for_ai/interim/val_sample_interactions_16407u_neg_seq.parquet")

    def init(self):
        self.notebook_persit_dp = os.path.abspath(f"data/{self.experiment_name}/{self.run_name}")

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            self.log_to_mlflow = False
            logger.warning("MLFlow is not enabled. Turn off tracking to Mlflow.")

        if self.log_to_mlflow:
            logger.info(
                f"Setting up Mlflow experiment: {self.experiment_name}, run_name: {self.run_name}"
            )

            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        if not self.testing:
            os.makedirs(self.notebook_persit_dp, exist_ok=True)
        return self
    
args = Args().init()
print(args.model_dump_json(indent=2))

2025-04-27 22:29:33.208 | WARNING  | __main__:init:43 - MLFlow is not enabled. Turn off tracking to Mlflow.


{
  "testing": false,
  "log_to_mlflow": false,
  "experiment_name": "first-attempt",
  "run_name": "018-sasrec",
  "notebook_persit_dp": "c:\\Users\\Trieu\\OneDrive\\Desktop\\recsys\\real_time_recsys\\notebooks\\data\\first-attempt\\018-sasrec",
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "group_name": "seq-modelling",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 16,
  "lr": 0.0001,
  "l2_emb": 0.0001,
  "early_stopping_patience": 10,
  "device": "cpu",
  "num_epochs": 100,
  "max_len": 10,
  "dropout": 0.5,
  "hidden_units": 128,
  "num_blocks": 1,
  "num_heads": 2,
  "num_workers": 4,
  "gbce_t": 0.0,
  "pad_token": 0,
  "train_data_fp": "c:\\Users\\Trieu\\OneDrive\\Desktop\\recsys\\real_time_recsys\\data_for_ai\\interim\\train_sample_interactions_16407u_neg_seq.parquet",
  "val_data_fp": "c:\\Users\\Trieu\\OneDrive\\Desktop\\recsys\\real_time_recsys\\data_for_ai\\interim\\val_sample_interactions_16407u_neg_seq.

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\pydantic\main.py:477: UserWarning:

Pydantic serializer warnings:
  Expected `int` but got `float` with value `0.0` - serialized value may not be as expected



In [5]:
train_df = pd.read_parquet(args.train_data_fp)
# convert train_df["rating"] to 1 if rating > 0 else 0
train_df[args.rating_col] = train_df[args.rating_col].apply(lambda x: 1 if x > 0 else 0)            
# train_df["item_sequence"] = train_df["item_sequence"].apply(lambda x: list(x))

val_df = pd.read_parquet(args.val_data_fp)
# convert val_df["rating"] to 1 if rating > 0 else 0
val_df[args.rating_col] = val_df[args.rating_col].apply(lambda x: 1 if x > 0 else 0)

assert set(val_df[args.user_col].unique()).issubset(set(train_df[args.user_col].unique())), "Validation users must be present in training users."

assert set(val_df[args.item_col].unique()).issubset(set(train_df[args.item_col].unique())), "Validation items must be present in training items."
assert train_df[args.timestamp_col].max() < val_df[args.timestamp_col].min(), "Validation data must be after training data. Otherwise, its a data contamination problem."

In [6]:
val_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
0,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B004FV4ROA,1,2020-12-27 00:30:31.146,11295,528,"[1898, 3479, 3908, 1570, 91, 2723, 2962, 106, ..."
1,AGSP5XAQPQBUUXZHEZSC65FD7NOQ,B07KFQFDNB,0,2020-12-27 00:30:31.146,11295,3503,"[3479, 3908, 1570, 91, 2723, 2962, 106, 3557, ..."
2,AEHS7YR7BGGWMZS24H5UR5IP46HQ,B08F1P3BCC,1,2020-12-27 01:44:52.242,1784,3925,"[4319, 3382, 4330, 1173, 1330, 423, 2868, 3167..."
3,AEHS7YR7BGGWMZS24H5UR5IP46HQ,B00HXT8EKE,0,2020-12-27 01:44:52.242,1784,1507,"[3382, 4330, 1173, 1330, 423, 2868, 3167, 1071..."
4,AGAVHCK42EGMVS7DGPRX6HBCUCNQ,B09Q3NR84W,1,2020-12-27 02:25:48.357,9042,4273,"[1311, 1416, 455, 3743, 1823, 2694, 3612, 3462..."
...,...,...,...,...,...,...,...
6953,AEEQZRQBOFHFBFPYBX2BZ5WOI33A,B01A08E70K,0,2022-02-19 16:56:53.030,1396,2441,"[3451, 3827, 1839, 1347, 2504, 2694, 4546, 427..."
6954,AHLN6GKTKZE22AON34YAQXTGK63A,B0C682GZ5X,1,2022-02-19 17:28:55.519,14550,4772,"[2950, 1812, 4735, 4165, 4575, 2440, 607, 4807..."
6955,AHLN6GKTKZE22AON34YAQXTGK63A,B09SWWCN6Q,0,2022-02-19 17:28:55.519,14550,4303,"[1812, 4735, 4165, 4575, 2440, 607, 4807, 374,..."
6956,AEMYBWDN67IB5IBTMHLHN76V4QHQ,B091K4WYD1,1,2022-02-19 22:08:53.253,2446,4086,"[644, 3602, 4569, 1865, 3030, 3653, 3803, 3998..."


In [7]:
train_df.head(3)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
0,AFZ4EK2LJ655XQKTEUELCARO6RYA,B00002EQCW,1,2003-01-23 03:28:15,8071,4,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
1,AFZ4EK2LJ655XQKTEUELCARO6RYA,B095JX15XF,0,2003-01-23 03:28:15,8071,4132,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
2,AFY2C4YOUP2SSMM43HD2L3FIEFZA,B00008SCFL,1,2003-11-25 18:12:09,7935,36,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."


In [8]:
def init_model(n_user, n_items, dropout, hidden_units, num_blocks, num_heads):
    """
    Initialize the model with the given parameters.
    """
    model = SASRec(
        user_num = n_user,
        item_num = n_items,
        dropout_rate = dropout,
        hidden_units = hidden_units,
        num_blocks = num_blocks,
        num_heads = num_heads,
    )
    return model

In [9]:
batch_size = 2
hidden_units = 8
dropout = 0.2
num_blocks = 1
num_heads = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 4, 5, 3, 0]

user_num = len(set(user_indices))
item_num = len(set(item_indices))

train_test_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
    }
)

model = init_model(user_num, item_num, dropout,hidden_units, num_blocks, num_heads)

# Example forward pass
model.eval()
user = torch.tensor([[0]])
seq = torch.tensor([[0,0,0,0,0,1,2,3,4,5]])
target_item = torch.tensor([[2]])
predictions = model.predict(user, seq, target_item)
print(predictions)

tensor([[6.4076e-06]], grad_fn=<SigmoidBackward0>)


In [10]:
print(type(train_df["item_sequence"][0]))


<class 'numpy.ndarray'>


In [11]:
train_df["item_indice"].max() + 1

4817

In [12]:
rating_dataset = SASRecDataset(
    train_df, "user_indice", "item_sequence", "item_indice", "rating",args.max_len, args.pad_token, args.timestamp_col, 
)
val_rating_dataset = SASRecDataset(
    val_df, "user_indice", "item_sequence", "item_indice", "rating", args.max_len, args.pad_token, args.timestamp_col, 
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [13]:
for i in train_loader:
    print(i["user"])
    print(i["sequence"])
    print(i["item"])
    print(i["rating"])
    break

tensor([ 7532,  4854,  9838,  4716, 13375,  8377,  1211,   696,  3542,  6313,
        15537, 13881,  4638,    10, 14494, 12842])
tensor([[4817, 4817, 4817, 4817, 2285, 2341, 1577,  746, 4040, 1983],
        [1678, 4564, 1293, 3571, 4659, 3578, 4653, 2629, 4450, 2071],
        [1633,  731, 1244,  159, 3688, 3330, 1211, 2922, 2216, 1267],
        [4492, 1134, 4276, 2485, 2412, 3571, 1091, 3585, 2410, 4516],
        [4817, 4817, 4817, 4817, 4817, 4817, 4817, 4817, 4817, 1502],
        [ 557, 1236, 1172, 1880, 2609,  967,  388, 3414, 2610,  766],
        [4817, 4817, 4817, 4817, 4817, 1236, 1885, 1404, 3541, 3533],
        [4817, 4817, 4817, 4817, 4817, 4817, 4817, 4817, 4817, 3436],
        [4817, 4817, 4817, 2163,  644,  223,  195, 4632,  127,  626],
        [4817, 4817, 4817, 4817, 4817,  446, 1533,  228, 4611, 3923],
        [4817, 4817, 4817, 4817, 4817, 4817,   12,  154, 1398, 1103],
        [4817, 4817, 4817, 4817, 4817, 4817, 3455, 1010, 4314, 2390],
        [1785, 1425, 2699, 2411

In [14]:
item_indices = train_df[args.item_col].unique()
user_indices = train_df[args.user_col].unique()
n_items = len(item_indices)
n_users = len(user_indices)

logger.info(f"Number of users: {n_users}, Number of items: {n_items}")
model = init_model(n_users, n_items, args.dropout, args.hidden_units, args.num_blocks, args.num_heads)

2025-04-27 22:29:38.935 | INFO     | __main__:<module>:6 - Number of users: 16407, Number of items: 4817


In [15]:
model

SASRec(
  (item_emb): Embedding(4818, 128, padding_idx=0)
  (pos_emb): Embedding(10, 128)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (attention_layernorms): ModuleList(
    (0): LayerNorm((128,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0): LayerNorm((128,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0): PointWiseFeedForward(
      (conv1): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.5, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      (dropout2): Dropout(p=0.5, inplace=False)
    )
  )
  (final_layer): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [16]:
idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
idm = IDMapper().load(idm_path)
idm.get_user_id(1)

'AE227WAM4NWQPJI33OPN7ZARNNZQ'

In [17]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.dropout, args.hidden_units, args.num_blocks, args.num_heads)
lit_model = SASRecLitModule(
    model,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    lr=args.lr,
    l2_emb=args.l2_emb,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=300,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | SASRec | 717 K  | train
-----------------------------------------
717 K     Trainable params
0         Non-trainable params
717 K     Total params
2.871     Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:252: UserWarning:

You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:252: UserWarning:

You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.

c:\Use

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-04-27 22:30:25.151 | INFO     | src.algo.gSASRec.trainer:on_fit_end:131 - Logging classification metrics...
2025-04-27 22:32:06.586 | INFO     | src.algo.gSASRec.trainer:on_fit_end:134 - Logging ranking metrics...


IndexError: index out of range in self